# Watson Assistant - Intents Association Rules

Omar Megawer

In [ ]:
!pip install update watson-developer-cloud

In [32]:
import pandas as pd
import numpy as np
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
import openpyxl
from watson_developer_cloud import ConversationV1
import datetime, dateutil.parser

Extracting the logs of the Workspace

In [114]:
ctx = {
  #"url": "https://gateway-fra.watsonplatform.net/assistant/api",
  "username": "",
  "password": ""
}

workspace_id = "bf194803-cdb5-406d-8763-55f954b74be5"

conversation = ConversationV1( username=ctx.get('username'), password=ctx.get('password'), version='2018-09-20')

extractedLogs = []
response1 = { 'pagination': 'DUMMY' } #This is set to start the loop
cursor = None
counter = 0
fltr = None  # Can be used to get a specific date

while response1['pagination']:
    print ("Reading Logs for Page: ", counter)
    counter = counter + 1
    response1 = conversation.list_logs(workspace_id=workspace_id, page_limit = 1000, cursor=cursor)
    extractedLogs.append(response1['logs'])
    
    #The API has a limit of a 100 logs to pull
    #To get the full log You need to update the cursor variable with a new cursor pointing to the next page
    #The cursor is fetched using the next_url variable
    
    if 'pagination' in response1 and 'next_url' in response1['pagination']:
        p = response1['pagination']['next_url']
        u = urlparse(p) 
        query = parse_qs(u.query)
        cursor = query['cursor'][0]
        
extraction = []
for logs in extractedLogs:
    for elem in logs:
        logID = elem['response']['context']['conversation_id']
        if not elem['response']['intents']:
            intentss = [{'confidence': 0, 'intent': 'No Intent Detected'}]
            intentss = intentss[0]
        else:
            intentss = elem['response']['intents'][0]
        
        row = {
            'Log_ID': logID,
            'Intent_Detected': intentss['intent']
        }
        extraction.append(row)
dataset = pd.DataFrame(extraction, columns=['Log_ID', 'Intent_Detected'])


Reading Logs for Page:  0


In [115]:
dataset

In [116]:
clean = dataset
clean = clean[clean.Intent_Detected != 'No Intent Detected'] # Just to remove biases 
clean = clean[clean.Intent_Detected != 'AFFIRMATIVE'] # Just to remove biases 
clean = clean[clean.Intent_Detected != 'NEGATIVE'] # Just to remove biases 
clean = clean[clean.Intent_Detected != 'Help_Connect_to_Agent']
clean = clean.reset_index()

clean = clean.groupby('Log_ID')['Intent_Detected'].apply(list)

Using Apriori to find associations between intents

In [117]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(clean).transform(clean)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

In [118]:
from mlxtend.frequent_patterns import apriori

F = apriori(df, min_support=0.1, use_colnames=True)

In [119]:
from mlxtend.frequent_patterns import association_rules

association_rules(F, metric="confidence", min_threshold=0.6)

Building a Recommender 

Preparing the data

In [120]:
import turicreate as tc

In [121]:
I = dataset
I = I[I.Intent_Detected != 'No Intent Detected'] # Just to remove biases 
I = I[I.Intent_Detected != 'AFFIRMATIVE'] # Just to remove biases 
I = I[I.Intent_Detected != 'NEGATIVE'] # Just to remove biases 
I = I[I.Intent_Detected != 'Help_Connect_to_Agent']
I = tc.SFrame(I)

Data Split

In [123]:
model = tc.recommender.create(I,'Log_ID' ,'Intent_Detected')

Recsys training: model = item_similarity

Preparing data set.

Data has 32 observations with 18 users and 5 items.

Data prepared in: 0.00608s

Training model from provided data.

...

| 2.974ms                             | 100              | 5               |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.004685s

In [124]:
results = model.recommend(k=1)

In [125]:
R = results.to_dataframe()
R

In [128]:
Intents_Asked =   ['Customer_Care_Store_Hours']

In [129]:
Intents_Asked

['Customer_Care_Store_Hours']

In [130]:
similar_items = model.get_similar_items(Intents_Asked, k=1)
similar_items.to_dataframe()